In [2]:
%pip install htmldate
%pip install twython
%pip install newspaper3k
%pip install lxml_html_clean
%pip install undetected-chromedriver selenium webdriver_manager
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Standard libraries
import time
import random
import re
import warnings
import asyncio
from urllib.parse import urlparse, parse_qs, unquote
from concurrent.futures import ThreadPoolExecutor
from dateutil.parser import parse

# Data processing
import pandas as pd
import aiohttp

# Web scraping
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Financial data
import yfinance as yf

# Suppress warnings
warnings.filterwarnings('ignore')

In [ ]:
chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--headless")  
chrome_prefs = {
    "profile.managed_default_content_settings.images": 2,      
    "profile.default_content_setting_values.stylesheets": 2,   
    "profile.default_content_setting_values.fonts": 2,         
}
chrome_options.add_experimental_option("prefs", chrome_prefs)
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=chrome_options)

driver.execute_cdp_cmd("Network.enable", {})
driver.execute_cdp_cmd("Network.setBlockedURLs", {
    "urls": ["*.png", "*.jpg", "*.jpeg", "*.gif", "*.css", "*.woff", "*.ttf"]
})

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


{}

In [ ]:
def get_set_newslinks():
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-quote-news-contanier"))
    )
    
    old_position = driver.execute_script("return window.pageYOffset;")
    new_position = None
    while new_position != old_position:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        new_position = driver.execute_script("return window.pageYOffset;")
        if new_position == old_position:
            break
        old_position = new_position

    news_cards = driver.find_elements(By.CSS_SELECTOR, "div.card-quote-news-contanier")
    news_links = []
    
    for card in news_cards:
        try:
            share_anchor = card.find_element(By.XPATH, ".//a[contains(@href, 'share?url=')]")
            share_href = share_anchor.get_attribute('href')
            # print(f"Share href: {share_href}")  # Debugging line to check the href value
            if share_href:
                parsed = urlparse(share_href)
                qs = parse_qs(parsed.query)
                if "url" in qs:
                    # ดึง URL จริงจาก query parameter 'url'
                    news_url = unquote(qs["url"][0])
                    # ถ้ามี query parameter 'symbol' ก็ให้ต่อเข้ากับ URL
                    if "symbol" in qs:
                        news_url += "&symbol=" + qs["symbol"][0]
                    news_links.append(news_url)
        except Exception as e:
            print(f"Error processing card: {e}")
            continue

    return news_links if news_links else []

In [ ]:
def click_next_page():
    """
    คลิกปุ่ม "Next" เพื่อไปหน้าถัดไปใน pagination
    คืนค่า True หากคลิกสำเร็จ, False หากไม่พบหรือเกิด error (อาจเป็นหน้าสุดท้าย)
    """
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "li.page-link__next > button"))
        )
        next_button.click()
        return True
    except Exception as e:
        print(f"Error clicking next page: {e}")
        return False

In [ ]:
start_date = "2024-10-10"
end_date   = "2025-04-10"

total_pages = 905  

base_url = f"https://www.set.or.th/th/market/news-and-alert/news?source=all&fromDate={start_date}&toDate={end_date}"
driver.get(base_url)
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card-quote-news-contanier"))
)

all_set_news_links = []

for page in range(1, total_pages + 1):
    print(f"Processing SET page {page} ...")
    links = get_set_newslinks()
    if len(links) > 0:
        all_set_news_links.extend(links)
        print(f"พบ {len(links)} ลิงก์ข่าวในหน้า {page}")
    else:
        print(f"ไม่พบข่าวในหน้า {page} จึงข้ามไปหน้าอื่น")
    
    if page < total_pages:
        if not click_next_page():
            print("ไม่สามารถคลิกปุ่ม Next ได้อีกต่อไป (อาจเป็นหน้าสุดท้าย)")
            break

driver.quit()

Processing SET page 1 ...
พบ 20 ลิงก์ข่าวในหน้า 1
Processing SET page 2 ...
พบ 20 ลิงก์ข่าวในหน้า 2
Processing SET page 3 ...
พบ 20 ลิงก์ข่าวในหน้า 3
Processing SET page 4 ...
พบ 20 ลิงก์ข่าวในหน้า 4
Processing SET page 5 ...
พบ 20 ลิงก์ข่าวในหน้า 5
Processing SET page 6 ...
พบ 20 ลิงก์ข่าวในหน้า 6
Processing SET page 7 ...
พบ 20 ลิงก์ข่าวในหน้า 7
Processing SET page 8 ...
พบ 20 ลิงก์ข่าวในหน้า 8
Processing SET page 9 ...
พบ 20 ลิงก์ข่าวในหน้า 9
Processing SET page 10 ...
พบ 20 ลิงก์ข่าวในหน้า 10
Processing SET page 11 ...
พบ 20 ลิงก์ข่าวในหน้า 11
Processing SET page 12 ...
พบ 20 ลิงก์ข่าวในหน้า 12
Processing SET page 13 ...
พบ 20 ลิงก์ข่าวในหน้า 13
Processing SET page 14 ...
พบ 20 ลิงก์ข่าวในหน้า 14
Processing SET page 15 ...
พบ 20 ลิงก์ข่าวในหน้า 15
Processing SET page 16 ...
พบ 20 ลิงก์ข่าวในหน้า 16
Processing SET page 17 ...
พบ 20 ลิงก์ข่าวในหน้า 17
Processing SET page 18 ...
พบ 20 ลิงก์ข่าวในหน้า 18
Processing SET page 19 ...
พบ 20 ลิงก์ข่าวในหน้า 19
Processing SET page 20 ...
พบ 

KeyboardInterrupt: 

In [ ]:
print("Total links", len(all_set_news_links))
for link in all_set_news_links:
    print(link)

รวมลิงก์ข่าวทั้งหมดที่พบ: 100
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95628601&symbol=MGC
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95636001&symbol=PTECH
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635901&symbol=ASK
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635701&symbol=NER
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635801&symbol=DIMET
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95620501&symbol=B
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635601&symbol=TSR
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95624701&symbol=AXTRART
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635401&symbol=MILL
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95634901&symbol=HMPRO
https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95634801&symbol=SNNP
https://www.set.or.th/th/market/news-and-alert/newsdetails

In [177]:
#openfile .txt to array
with open("set_news_links copy.txt", "r", encoding="utf-8") as file:
    all_set_news_links_test = [line.strip() for line in file if line.strip()]

In [ ]:
# -------- Configuration --------
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "th-TH,th;q=0.9,en-US;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",  
}
CONCURRENT_REQUESTS = 200
TIMEOUT_SECONDS = 30
MAX_RETRIES = 3

sem = asyncio.Semaphore(CONCURRENT_REQUESTS)
executor = ThreadPoolExecutor(max_workers=CONCURRENT_REQUESTS // 4)

def parse_article(html: str, url: str, params: dict):
    """CPU-bound parsing ที่จะรันบน thread pool"""
    soup = BeautifulSoup(html, "lxml")
    title_tag = soup.select_one("div.title h2")
    title = title_tag.get_text(strip=True) if title_tag else ""
    dt = soup.select_one("span.mb-0.fs-14px.default-font-family span.ms-2")
    if dt:
        parts = dt.get_text(strip=True).split()
        publish_date = " ".join(parts[:-1])
        publish_time = parts[-1]
    else:
        publish_date = publish_time = ""
    body = soup.select_one("pre.raw-html-new div.raw-html")
    body_text = body.get_text(strip=True) if body else ""
    return {
        "url": url,
        "id": params["id"],
        "symbol": params["symbol"],
        "title": title,
        "publish_date": publish_date,
        "publish_time": publish_time,
        "body_text": body_text
    }

async def fetch_article(session, url):
    qs = parse_qs(urlparse(url).query)
    params = {"id": qs.get("id", [""])[0], "symbol": qs.get("symbol", [""])[0]}

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            async with sem:
                async with session.get(
                    "https://www.set.or.th/th/market/news-and-alert/newsdetails",
                    params=params, headers=HEADERS
                ) as resp:
                    resp.raise_for_status()
                    raw = await resp.read()
                    html = raw.decode("utf-8", errors="ignore")

            loop = asyncio.get_running_loop()
            return await loop.run_in_executor(
                executor, parse_article, html, url, params
            )

        except (aiohttp.ClientError, asyncio.TimeoutError) as e:
            if attempt < MAX_RETRIES:
                backoff = 2 ** (attempt - 1) + random.random()
                await asyncio.sleep(backoff)
            else:
                print(f"[Failed] {url} => {e}")
                return None

async def scrape_all(urls):
    connector = aiohttp.TCPConnector(
        limit=CONCURRENT_REQUESTS,
        limit_per_host=CONCURRENT_REQUESTS // 2
    )
    timeout = aiohttp.ClientTimeout(total=TIMEOUT_SECONDS)

    async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
        tasks = [fetch_article(session, u) for u in urls]
        results = await asyncio.gather(*tasks)
    good = [r for r in results if r]
    return pd.DataFrame(good)

all_urls = all_set_news_links  
start = time.perf_counter()
article_data = await scrape_all(all_urls)
elapsed = time.perf_counter() - start
print(f"Scraped {len(article_data)} / {len(all_urls)} articles in {elapsed:.1f}s")


Scraped 18100 / 18100 articles in 394.5s


In [ ]:
# Show DataFrame of article sentiments data

article_data

,url,id,symbol,title,publish_date,publish_time,body_text
0,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,10 Apr 2025,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...
1,https://www.set.or.th/th/market/news-and-alert...,95636001,PTECH,ชี้แจงข่าวหรือข้อมูลตามที่ตลาดหลักทรัพย์ฯ สอบถาม,10 Apr 2025,22:24:00,ตลาดหลักทรัพย์ฯ ขอให้ชี้แจงข่าวหรือข้อมูล ...
2,https://www.set.or.th/th/market/news-and-alert...,95635901,ASK,แจ้งมติการประชุมใหญ่สามัญประจำปีผู้ถือหุ้น,10 Apr 2025,22:22:00,มติที่ประชุมผู้ถือหุ้น ...
3,https://www.set.or.th/th/market/news-and-alert...,95635701,NER,แจ้งมติการประชุมสามัญผู้ถือหุ้นประจำปี 2568,10 เม.ย. 2568,21:45:00,มติที่ประชุมผู้ถือหุ้น ...
4,https://www.set.or.th/th/market/news-and-alert...,95635801,DIMET,แจ้งมติการประชุมสามัญผู้ถือหุ้นประจำปี 2568,10 เม.ย. 2568,21:37:00,ข้อสงวนสิทธิ์สารสนเทศฉบับนี้จัดทำและเผยแพร่โดย...
...,...,...,...,...,...,...,...
18095,https://www.set.or.th/th/market/news-and-alert...,95620701,STOWER,แจ้งความคืบหน้าการให้ความช่วยเหลือทางการเงินแก...,10 เม.ย. 2568,19:15:00,ข้อสงวนสิทธิ์สารสนเทศฉบับนี้จัดทำและเผยแพร่โดย...
18096,https://www.set.or.th/th/market/news-and-alert...,95634501,MILL,การกำหนดวันประชุม ระเบียบวาระการประชุมสำหรับกา...,10 เม.ย. 2568,18:59:00,กำหนดการประชุมผู้ถือหุ้น \t...
18097,https://www.set.or.th/th/market/news-and-alert...,95634401,JMART,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,10 เม.ย. 2568,18:59:00,มติที่ประชุมผู้ถือหุ้น ...
18098,https://www.set.or.th/th/market/news-and-alert...,95634201,JMT,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,10 เม.ย. 2568,18:57:00,มติที่ประชุมผู้ถือหุ้น ...


In [180]:
all_set_news_links

['https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95628601&symbol=MGC',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95636001&symbol=PTECH',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635901&symbol=ASK',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635701&symbol=NER',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635801&symbol=DIMET',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95620501&symbol=B',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635601&symbol=TSR',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95624701&symbol=AXTRART',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95635401&symbol=MILL',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95634901&symbol=HMPRO',
 'https://www.set.or.th/th/market/news-and-alert/newsdetails?id=95634801&symbol=SNNP',
 'https://www.set.or.th/th/market/news-and-a

In [181]:
# Save DataFrame 

article_data.to_pickle("set_news_articles.pkl")

In [182]:
article_data.to_csv("set_news_articles.csv", index=False, encoding='utf-8-sig')

In [183]:
# Save URLS to text file

with open("set_news_links.txt", "w", encoding='utf-8') as f:
    for link in all_set_news_links:
        f.write("%s\n" % link)

In [4]:
article_sentiments = pd.read_pickle("set_news_articles.pkl")

In [5]:
article_sentiments

,url,id,symbol,title,publish_date,publish_time,body_text
0,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,10 Apr 2025,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...
1,https://www.set.or.th/th/market/news-and-alert...,95636001,PTECH,ชี้แจงข่าวหรือข้อมูลตามที่ตลาดหลักทรัพย์ฯ สอบถาม,10 Apr 2025,22:24:00,ตลาดหลักทรัพย์ฯ ขอให้ชี้แจงข่าวหรือข้อมูล ...
2,https://www.set.or.th/th/market/news-and-alert...,95635901,ASK,แจ้งมติการประชุมใหญ่สามัญประจำปีผู้ถือหุ้น,10 Apr 2025,22:22:00,มติที่ประชุมผู้ถือหุ้น ...
3,https://www.set.or.th/th/market/news-and-alert...,95635701,NER,แจ้งมติการประชุมสามัญผู้ถือหุ้นประจำปี 2568,10 เม.ย. 2568,21:45:00,มติที่ประชุมผู้ถือหุ้น ...
4,https://www.set.or.th/th/market/news-and-alert...,95635801,DIMET,แจ้งมติการประชุมสามัญผู้ถือหุ้นประจำปี 2568,10 เม.ย. 2568,21:37:00,ข้อสงวนสิทธิ์สารสนเทศฉบับนี้จัดทำและเผยแพร่โดย...
...,...,...,...,...,...,...,...
18095,https://www.set.or.th/th/market/news-and-alert...,95620701,STOWER,แจ้งความคืบหน้าการให้ความช่วยเหลือทางการเงินแก...,10 เม.ย. 2568,19:15:00,ข้อสงวนสิทธิ์สารสนเทศฉบับนี้จัดทำและเผยแพร่โดย...
18096,https://www.set.or.th/th/market/news-and-alert...,95634501,MILL,การกำหนดวันประชุม ระเบียบวาระการประชุมสำหรับกา...,10 เม.ย. 2568,18:59:00,กำหนดการประชุมผู้ถือหุ้น \t...
18097,https://www.set.or.th/th/market/news-and-alert...,95634401,JMART,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,10 เม.ย. 2568,18:59:00,มติที่ประชุมผู้ถือหุ้น ...
18098,https://www.set.or.th/th/market/news-and-alert...,95634201,JMT,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,10 เม.ย. 2568,18:57:00,มติที่ประชุมผู้ถือหุ้น ...


In [94]:
thai_months = {
    'ม.ค.': 1, 'ก.พ.': 2, 'มี.ค.': 3, 'เม.ย.': 4, 'พ.ค.': 5, 'มิ.ย.': 6,
    'ก.ค.': 7, 'ส.ค.': 8, 'ก.ย.': 9, 'ต.ค.': 10, 'พ.ย.': 11, 'ธ.ค.': 12
}

def parse_date_mixed(s):
    if pd.isna(s):
        return pd.NaT
    s = str(s).strip()
    for th_abbr, month in thai_months.items():
        if th_abbr in s:
            day_str, mon_str, year_str = s.split()
            day = int(day_str)
            year_be = int(year_str)
            year_ce = year_be - 543
            return pd.Timestamp(year=year_ce, month=month, day=day)
    try:
        return parse(s, dayfirst=True)
    except:
        return pd.NaT



df_news = article_sentiments.copy()

# Apply parsing
df_news['publish_date'] = df_news['publish_date'].apply(parse_date_mixed)
df_news['publish_time'] = pd.to_datetime(df_news['publish_time'], format='%H:%M:%S').dt.time


def to_yf_ticker(sym):
    sym = str(sym).strip().upper()
    m = re.match(r'^([A-Z]+)(\d*)$', sym)
    if not m:
        return None
    letters, digits = m.groups()
    return f"{letters}{digits}.BK"

df_news['yfinance_ticker'] = df_news['symbol'].apply(to_yf_ticker)
df_news = df_news.dropna(subset=['yfinance_ticker'])

In [95]:
df_news

,url,id,symbol,title,publish_date,publish_time,body_text,yfinance_ticker
0,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...,MGC.BK
1,https://www.set.or.th/th/market/news-and-alert...,95636001,PTECH,ชี้แจงข่าวหรือข้อมูลตามที่ตลาดหลักทรัพย์ฯ สอบถาม,2025-04-10,22:24:00,ตลาดหลักทรัพย์ฯ ขอให้ชี้แจงข่าวหรือข้อมูล ...,PTECH.BK
2,https://www.set.or.th/th/market/news-and-alert...,95635901,ASK,แจ้งมติการประชุมใหญ่สามัญประจำปีผู้ถือหุ้น,2025-04-10,22:22:00,มติที่ประชุมผู้ถือหุ้น ...,ASK.BK
3,https://www.set.or.th/th/market/news-and-alert...,95635701,NER,แจ้งมติการประชุมสามัญผู้ถือหุ้นประจำปี 2568,2025-04-10,21:45:00,มติที่ประชุมผู้ถือหุ้น ...,NER.BK
4,https://www.set.or.th/th/market/news-and-alert...,95635801,DIMET,แจ้งมติการประชุมสามัญผู้ถือหุ้นประจำปี 2568,2025-04-10,21:37:00,ข้อสงวนสิทธิ์สารสนเทศฉบับนี้จัดทำและเผยแพร่โดย...,DIMET.BK
...,...,...,...,...,...,...,...,...
18095,https://www.set.or.th/th/market/news-and-alert...,95620701,STOWER,แจ้งความคืบหน้าการให้ความช่วยเหลือทางการเงินแก...,2025-04-10,19:15:00,ข้อสงวนสิทธิ์สารสนเทศฉบับนี้จัดทำและเผยแพร่โดย...,STOWER.BK
18096,https://www.set.or.th/th/market/news-and-alert...,95634501,MILL,การกำหนดวันประชุม ระเบียบวาระการประชุมสำหรับกา...,2025-04-10,18:59:00,กำหนดการประชุมผู้ถือหุ้น \t...,MILL.BK
18097,https://www.set.or.th/th/market/news-and-alert...,95634401,JMART,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:59:00,มติที่ประชุมผู้ถือหุ้น ...,JMART.BK
18098,https://www.set.or.th/th/market/news-and-alert...,95634201,JMT,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:57:00,มติที่ประชุมผู้ถือหุ้น ...,JMT.BK


In [108]:
start_date = "2024-10-03"
end_date   = "2025-04-17"

def fetch_ohlcv(tickers, start_date, end_date, interval="1d"):

    df = yf.download(
        tickers=tickers,
        start=start_date,
        end=end_date,
        interval=interval,
        group_by="ticker",   
        auto_adjust=False,
        progress=False
    )
    
    if isinstance(df.columns, pd.MultiIndex):
        frames = []
        for tk in tickers:
            if tk in df.columns.levels[0]:
                tmp = df[tk].copy()
                tmp = tmp.rename(columns={
                    "Open": "open", "High": "high", "Low": "low",
                    "Close": "close", "Volume": "volume"
                })
                tmp = tmp.reset_index()
                tmp["ticker"] = tk
                frames.append(tmp[["Date", "ticker", "open", "high", "low", "close", "volume"]])
        result = pd.concat(frames, ignore_index=True)
    else:
        result = df.rename(columns={
            "Open": "open", "High": "high", "Low": "low",
            "Close": "close", "Volume": "volume"
        }).reset_index()
        result["ticker"] = tickers[0]
        result = result[["Date", "ticker", "open", "high", "low", "close", "volume"]]
    
    result = result.dropna(subset=["close"])
    result = result.sort_values(["ticker", "Date"]).reset_index(drop=True)
    return result


In [109]:
tickers = df_news['yfinance_ticker'].unique().tolist()
df_price = fetch_ohlcv(tickers, start_date, end_date)
print(df_price.head())

Price       Date  ticker  open  high   low  close    volume
0     2024-10-03  ASK.BK  13.5  13.5  13.2   13.2  678800.0
1     2024-10-04  ASK.BK  13.2  13.2  13.2   13.2       0.0
2     2024-10-07  ASK.BK  13.1  13.3  13.0   13.1  346500.0
3     2024-10-08  ASK.BK  13.2  13.4  13.1   13.3  429400.0
4     2024-10-09  ASK.BK  13.2  13.5  13.2   13.3  335100.0


In [110]:
df_price = df_price.dropna(subset=['close','ticker'])

df_price['Date'] = pd.to_datetime(df_price['Date'])
df_price = df_price.sort_values(['ticker','Date']).reset_index(drop=True)

In [111]:
df_price

Price,Date,ticker,open,high,low,close,volume
0,2024-10-03,ASK.BK,13.50,13.50,13.20,13.20,678800.0
1,2024-10-04,ASK.BK,13.20,13.20,13.20,13.20,0.0
2,2024-10-07,ASK.BK,13.10,13.30,13.00,13.10,346500.0
3,2024-10-08,ASK.BK,13.20,13.40,13.10,13.30,429400.0
4,2024-10-09,ASK.BK,13.20,13.50,13.20,13.30,335100.0
...,...,...,...,...,...,...,...
2461,2025-04-08,TSR.BK,0.17,0.19,0.17,0.17,1580356.0
2462,2025-04-09,TSR.BK,0.15,0.18,0.15,0.18,658403.0
2463,2025-04-10,TSR.BK,0.18,0.18,0.16,0.18,362808.0
2464,2025-04-11,TSR.BK,0.17,0.18,0.17,0.18,25300.0


In [112]:
indicator_dfs = []
for tkr, grp in df_price.groupby('ticker', sort=False):
    grp = grp.sort_values('Date').copy()
    grp['ema12'] = grp['close'].ewm(span=12).mean()
    grp['ema26'] = grp['close'].ewm(span=26).mean()
    grp['MACD'] = grp['ema12'] - grp['ema26']
    grp['MACD_signal'] = grp['MACD'].ewm(span=9).mean()
    # RSI(14)
    delta = grp['close'].diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.rolling(window=14).mean()
    ma_down = down.rolling(window=14).mean()
    grp['RSI'] = 100 - (100 / (1 + ma_up/ma_down))
    # MA_short(5), MA_long(20)
    grp['MA_short'] = grp['close'].rolling(window=5).mean()
    grp['MA_long'] = grp['close'].rolling(window=20).mean()
    indicator_dfs.append(grp)

df_price_ind = pd.concat(indicator_dfs, ignore_index=True)

In [113]:
df_price_ind['Date'] = pd.to_datetime(df_price_ind['Date'])
df_price_ind = df_price_ind.sort_values(['ticker','Date']).reset_index(drop=True)

In [114]:
df_price_ind['next_close']   = df_price_ind.groupby('ticker')['close'].shift(-1)
df_price_ind['return_next']  = df_price_ind['next_close']/df_price_ind['close'] - 1

In [115]:
df_price_ind

Price,Date,ticker,open,high,low,close,volume,ema12,ema26,MACD,MACD_signal,RSI,MA_short,MA_long,next_close,return_next
0,2024-10-03,ASK.BK,13.50,13.50,13.20,13.20,678800.0,13.200000,13.200000,0.000000,0.000000,NaN,NaN,NaN,13.20,0.000000
1,2024-10-04,ASK.BK,13.20,13.20,13.20,13.20,0.0,13.200000,13.200000,0.000000,0.000000,NaN,NaN,NaN,13.10,-0.007576
2,2024-10-07,ASK.BK,13.10,13.30,13.00,13.10,346500.0,13.160970,13.164071,-0.003101,-0.001271,NaN,NaN,NaN,13.30,0.015267
3,2024-10-08,ASK.BK,13.20,13.40,13.10,13.30,429400.0,13.204856,13.202071,0.002786,0.000103,NaN,NaN,NaN,13.30,0.000000
4,2024-10-09,ASK.BK,13.20,13.50,13.20,13.30,335100.0,13.230707,13.224781,0.005926,0.001835,NaN,13.220,NaN,13.00,-0.022556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,2025-04-08,TSR.BK,0.17,0.19,0.17,0.17,1580356.0,0.204222,0.229740,-0.025517,-0.022039,21.428578,0.186,0.2255,0.18,0.058824
2462,2025-04-09,TSR.BK,0.15,0.18,0.15,0.18,658403.0,0.200496,0.226055,-0.025559,-0.022743,28.571438,0.184,0.2215,0.18,0.000000
2463,2025-04-10,TSR.BK,0.18,0.18,0.16,0.18,362808.0,0.197343,0.222643,-0.025300,-0.023255,23.076931,0.182,0.2175,0.18,0.000000
2464,2025-04-11,TSR.BK,0.17,0.18,0.17,0.18,25300.0,0.194675,0.219484,-0.024810,-0.023566,23.076931,0.180,0.2135,0.17,-0.055556


In [116]:
df_price_ind = df_price_ind[df_price_ind['volume'] > 0]  \
    .dropna(subset=['ema12','ema26','MACD','MACD_signal','RSI','MA_short','MA_long','next_close','return_next']) \
    .reset_index(drop=True)

In [117]:
df_price_ind

Price,Date,ticker,open,high,low,close,volume,ema12,ema26,MACD,MACD_signal,RSI,MA_short,MA_long,next_close,return_next
0,2024-11-01,ASK.BK,12.20,12.40,12.10,12.10,151900.0,12.456236,12.629872,-0.173636,-0.113788,31.999994,12.220,12.8000,12.00,-0.008264
1,2024-11-04,ASK.BK,12.20,12.30,12.00,12.00,324200.0,12.383878,12.571648,-0.187770,-0.128722,30.769219,12.140,12.7400,11.80,-0.016667
2,2024-11-05,ASK.BK,12.00,12.00,11.80,11.80,539200.0,12.291715,12.501606,-0.209890,-0.145077,29.629624,12.020,12.6700,11.60,-0.016949
3,2024-11-06,ASK.BK,11.90,11.90,11.60,11.60,455600.0,12.182965,12.421110,-0.238145,-0.163801,27.586202,11.940,12.5950,11.10,-0.043103
4,2024-11-07,ASK.BK,11.50,11.50,11.00,11.10,810700.0,12.013276,12.304929,-0.291653,-0.189493,10.344809,11.720,12.4850,11.20,0.009009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,2025-04-04,TSR.BK,0.20,0.20,0.18,0.19,682300.0,0.210445,0.234519,-0.024074,-0.021170,25.000006,0.192,0.2300,0.17,-0.105263
2009,2025-04-08,TSR.BK,0.17,0.19,0.17,0.17,1580356.0,0.204222,0.229740,-0.025517,-0.022039,21.428578,0.186,0.2255,0.18,0.058824
2010,2025-04-09,TSR.BK,0.15,0.18,0.15,0.18,658403.0,0.200496,0.226055,-0.025559,-0.022743,28.571438,0.184,0.2215,0.18,0.000000
2011,2025-04-10,TSR.BK,0.18,0.18,0.16,0.18,362808.0,0.197343,0.222643,-0.025300,-0.023255,23.076931,0.182,0.2175,0.18,0.000000


In [118]:
df_news['publish_date'] = pd.to_datetime(df_news['publish_date'], errors='coerce')

df_news_clean = df_news.dropna(subset=['publish_date', 'yfinance_ticker'])

df_news_clean_sorted = df_news_clean.sort_values('publish_date')
df_price_ind = df_price_ind.rename(columns={'Date': 'publish_date', 'ticker': 'yfinance_ticker'})
df_price_ind['publish_date'] = pd.to_datetime(df_price_ind['publish_date'], errors='coerce')

df_price_ind_clean = df_price_ind.dropna(subset=['publish_date', 'yfinance_ticker'])

df_price_ind_clean_sorted = df_price_ind_clean.sort_values('publish_date').reset_index(drop=True)

df_merged = pd.merge_asof(
    df_news_clean_sorted,
    df_price_ind_clean_sorted,
    on='publish_date',
    by='yfinance_ticker',
    direction='backward'
)


In [119]:
def clean_body(text):
    if not isinstance(text, str):
        return ''
    text = re.split(r'_{5,}[\s\S]*$', text)[0]
    text = re.split(r'สารสนเทศฉบับนี้จัดทำและเผยแพร่[\s\S]*$', text)[0]
    text = re.split(r'ข้อสงวนสิทธิ์[\s\S]*$', text)[0]
    return text.strip()

df_merged['body_text'] = df_merged['body_text'].apply(clean_body)

df_merged_clean = df_merged.dropna(subset=['open', 'high', 'low', 'close', 'volume', 'ema12', 'ema26', 'MACD', 'MACD_signal', 'RSI', 'MA_short', 'MA_long'])

min_len = 10
mask = df_merged_clean['body_text'].str.len() >= min_len

df_merged_clean = df_merged_clean[mask].reset_index(drop=True)


In [120]:
df_merged_clean

,url,id,symbol,title,publish_date,publish_time,body_text,yfinance_ticker,open,high,...,volume,ema12,ema26,MACD,MACD_signal,RSI,MA_short,MA_long,next_close,return_next
0,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...,MGC.BK,3.02,3.10,...,264200.0,3.028064,3.053122,-0.025058,0.017917,34.615384,2.892,3.1370,2.96,-0.013333
1,https://www.set.or.th/th/market/news-and-alert...,95634401,JMART,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:59:00,มติที่ประชุมผู้ถือหุ้น ...,JMART.BK,7.70,7.75,...,10638500.0,8.307818,8.974740,-0.666922,-0.458398,12.698402,7.650,8.9975,7.15,-0.040268
2,https://www.set.or.th/th/market/news-and-alert...,95634201,JMT,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:57:00,มติที่ประชุมผู้ถือหุ้น ...,JMT.BK,12.70,12.80,...,12230700.0,12.729434,13.138030,-0.408595,-0.282165,26.190470,12.380,13.1650,11.60,-0.049180
3,https://www.set.or.th/th/market/news-and-alert...,95634001,J,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:56:00,มติที่ประชุมผู้ถือหุ้น ...,J.BK,0.86,0.89,...,202500.0,0.931639,0.969555,-0.037916,-0.027792,21.739134,0.898,0.9645,0.83,-0.045977
4,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...,MGC.BK,3.02,3.10,...,264200.0,3.028064,3.053122,-0.025058,0.017917,34.615384,2.892,3.1370,2.96,-0.013333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14475,https://www.set.or.th/th/market/news-and-alert...,95632801,BMSCG,รายงานมูลค่าสินทรัพย์สุทธิต่อหน่วยลงทุน,2025-04-10,19:21:00,รายงานมูลค่าสินทรัพย์สุทธิต่อหน่วยลงทุน ...,BMSCG.BK,6.78,6.81,...,3300.0,7.046470,7.311426,-0.264955,-0.177090,23.809515,6.756,7.3415,6.67,-0.020558
14476,https://www.set.or.th/th/market/news-and-alert...,95619601,ORI,คำอธิบายและวิเคราะห์ของฝ่ายจัดการ ประจำปี สิ้น...,2025-04-10,19:23:00,คำอธิบายและวิเคราะห์ของฝ่ายจัดการ \t\t...,ORI.BK,1.85,1.94,...,5560000.0,2.053234,2.312144,-0.258911,-0.228021,16.379305,1.864,2.2650,1.84,-0.021277
14477,https://www.set.or.th/th/market/news-and-alert...,95634801,SNNP,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,19:27:00,รายงานผลการซื้อหุ้นคืน \t\...,SNNP.BK,11.80,11.90,...,426300.0,11.816764,12.002670,-0.185907,-0.106085,30.000038,11.640,12.0650,11.60,0.000000
14478,https://www.set.or.th/th/market/news-and-alert...,95634901,HMPRO,แจ้งมติที่ประชุมสามัญผู้ถือหุ้นประจำปี 2568,2025-04-10,19:37:00,มติที่ประชุมผู้ถือหุ้น ...,HMPRO.BK,8.85,8.90,...,36565400.0,8.396658,8.304265,0.092393,0.028318,62.903218,8.470,8.1750,8.90,0.022988


In [121]:
df_merged_clean['label_binary'] = (df_merged_clean['return_next'] > 0).astype(int)

In [127]:
df_merged_clean

,url,id,symbol,title,publish_date,publish_time,body_text,yfinance_ticker,open,high,...,ema12,ema26,MACD,MACD_signal,RSI,MA_short,MA_long,next_close,return_next,label_binary
0,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...,MGC.BK,3.02,3.10,...,3.028064,3.053122,-0.025058,0.017917,34.615384,2.892,3.1370,2.96,-0.013333,0
1,https://www.set.or.th/th/market/news-and-alert...,95634401,JMART,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:59:00,มติที่ประชุมผู้ถือหุ้น ...,JMART.BK,7.70,7.75,...,8.307818,8.974740,-0.666922,-0.458398,12.698402,7.650,8.9975,7.15,-0.040268,0
2,https://www.set.or.th/th/market/news-and-alert...,95634201,JMT,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:57:00,มติที่ประชุมผู้ถือหุ้น ...,JMT.BK,12.70,12.80,...,12.729434,13.138030,-0.408595,-0.282165,26.190470,12.380,13.1650,11.60,-0.049180,0
3,https://www.set.or.th/th/market/news-and-alert...,95634001,J,แจ้งมติที่ประชุมสามัญผู้ถือหุ้น ประจำปี 2568,2025-04-10,18:56:00,มติที่ประชุมผู้ถือหุ้น ...,J.BK,0.86,0.89,...,0.931639,0.969555,-0.037916,-0.027792,21.739134,0.898,0.9645,0.83,-0.045977,0
4,https://www.set.or.th/th/market/news-and-alert...,95628601,MGC,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,22:29:00,รายงานผลการซื้อหุ้นคืน \t\...,MGC.BK,3.02,3.10,...,3.028064,3.053122,-0.025058,0.017917,34.615384,2.892,3.1370,2.96,-0.013333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14475,https://www.set.or.th/th/market/news-and-alert...,95632801,BMSCG,รายงานมูลค่าสินทรัพย์สุทธิต่อหน่วยลงทุน,2025-04-10,19:21:00,รายงานมูลค่าสินทรัพย์สุทธิต่อหน่วยลงทุน ...,BMSCG.BK,6.78,6.81,...,7.046470,7.311426,-0.264955,-0.177090,23.809515,6.756,7.3415,6.67,-0.020558,0
14476,https://www.set.or.th/th/market/news-and-alert...,95619601,ORI,คำอธิบายและวิเคราะห์ของฝ่ายจัดการ ประจำปี สิ้น...,2025-04-10,19:23:00,คำอธิบายและวิเคราะห์ของฝ่ายจัดการ \t\t...,ORI.BK,1.85,1.94,...,2.053234,2.312144,-0.258911,-0.228021,16.379305,1.864,2.2650,1.84,-0.021277,0
14477,https://www.set.or.th/th/market/news-and-alert...,95634801,SNNP,แบบรายงานผลการซื้อหุ้นคืนกรณีเพื่อการบริหารทาง...,2025-04-10,19:27:00,รายงานผลการซื้อหุ้นคืน \t\...,SNNP.BK,11.80,11.90,...,11.816764,12.002670,-0.185907,-0.106085,30.000038,11.640,12.0650,11.60,0.000000,0
14478,https://www.set.or.th/th/market/news-and-alert...,95634901,HMPRO,แจ้งมติที่ประชุมสามัญผู้ถือหุ้นประจำปี 2568,2025-04-10,19:37:00,มติที่ประชุมผู้ถือหุ้น ...,HMPRO.BK,8.85,8.90,...,8.396658,8.304265,0.092393,0.028318,62.903218,8.470,8.1750,8.90,0.022988,1


In [128]:
df_merged_clean['label_binary'].value_counts()

label_binary
0    13575
1      905
Name: count, dtype: int64

In [129]:
# Save DataFrame 

df_merged_clean.to_pickle("set_news_articles_final.pkl")